
<div dir="rtl" align="right">

# 📘 اسلات A-2: مدل‌های I/O و سوکت‌ها — Echo Server (Blocking & Async)

**هدف جلسه (۴۵ دقیقه):**
- شناخت عملی Socket (TCP) و تفاوت Blocking vs Async.
- ساخت Echo Server بلاکینگ (پورت 9000) و Async (پورت 9001).
- تست با PowerShell و مشاهدهٔ هم‌زمانی در نسخهٔ Async.

</div>



<div dir="rtl" align="right">

## ۱) Socket چیست؟
- endpoint شبکه با `(IP, Port)`. HTTP روی TCP socket سوار است.
- برای لمس I/O، echo-server می‌سازیم تا هرچه دریافت می‌کند، همان را برگرداند.

</div>



<div dir="rtl" align="right">

## ۲) نسخهٔ Blocking (پورت 9000)

</div>


In [ ]:
# blocking_echo.py
import socket

def run_blocking_echo(host: str = "127.0.0.1", port: int = 9000) -> None:
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    server.bind((host, port))
    server.listen()
    print(f"Blocking Echo Server running on {host}:{port} ... (Ctrl+C to stop)")

    while True:
        conn, addr = server.accept()
        print("Connected by", addr)
        try:
            while True:
                data = conn.recv(1024)
                if not data:
                    break
                conn.sendall(data)
        finally:
            conn.close()

if __name__ == "__main__":
    run_blocking_echo()


<div dir="rtl" align="right">

### تست Blocking با PowerShell
```
python blocking_echo.py
```
در ترمینال جدا:
```
$client = New-Object System.Net.Sockets.TcpClient("127.0.0.1", 9000)
$stream = $client.GetStream()
$writer = New-Object System.IO.StreamWriter($stream); $writer.AutoFlush = $true
$reader = New-Object System.IO.StreamReader($stream)

$writer.WriteLine("salam echo server!")
$reader.ReadLine()

$client.Close()
```

</div>



<div dir="rtl" align="right">

## ۳) نسخهٔ Async با asyncio (پورت 9001)

</div>


In [ ]:
# async_echo.py
import asyncio
from typing import Tuple

async def handle_echo(reader: asyncio.StreamReader, writer: asyncio.StreamWriter) -> None:
    addr: Tuple[str, int] = writer.get_extra_info("peername")
    print(f"Connected by {addr}")
    while True:
        data = await reader.readline()
        if not data:
            break
        writer.write(data)
        await writer.drain()
    writer.close()
    await writer.wait_closed()

async def main(host: str = "127.0.0.1", port: int = 9001) -> None:
    server = await asyncio.start_server(handle_echo, host, port)
    addrs = ", ".join(str(sock.getsockname()) for sock in server.sockets)
    print(f"Async Echo Server running on {addrs} ... (Ctrl+C to stop)")
    async with server:
        await server.serve_forever()

if __name__ == "__main__":
    asyncio.run(main())


<div dir="rtl" align="right">

### تست Async با PowerShell (چند پیام در یک اتصال)
```
python async_echo.py
```
در ترمینال جدا:
```
$client = New-Object System.Net.Sockets.TcpClient("127.0.0.1", 9001)
$stream = $client.GetStream()
$writer = New-Object System.IO.StreamWriter($stream); $writer.AutoFlush = $true
$reader = New-Object System.IO.StreamReader($stream)

$writer.WriteLine("line one");   $reader.ReadLine()
$writer.WriteLine("line two");   $reader.ReadLine()
$writer.WriteLine("LINE THREE"); $reader.ReadLine()

$client.Close()
```
**نکته:** برای curl در ویندوز از `curl.exe` استفاده کن.

</div>



<div dir="rtl" align="right">

## 🧪 تمرین‌ها
1) در نسخهٔ Async پیام را `UPPERCASE` برگردان.  
2) در Blocking بعد از هر echo اتصال را ببند. تغییر رفتاری را توضیح بده.  
3) دو ترمینال PowerShell هم‌زمان به Async وصل کن و موازی‌بودن را نشان بده (لاگ‌ها را کپی کن).

## ❓ سؤالات تشریحی
- تفاوت Blocking و Async در رسیدگی به چند اتصال چیست؟  
- چرا Async برای وب‌سرورهای مدرن مهم است؟  
- تفاوت `127.0.0.1` با `0.0.0.0` چیست؟

## ▶️ اسلات بعدی: A-3 — Asyncio عمیق‌تر
Event loop، Task، await/async، Cancellation/Timeout، `gather`/`shield`، و ربط مستقیم به FastAPI.

</div>
